# FieldTwin API - Python example plotting connection profile

This python notebook provides an example demo of using the FieldTwin API.   
See the API documentation (https://api.fieldtwin.com/).

The example creates an input form to enter the FieldTwin project and subproject     
and loads the subproject data using the FieldTwin API.

A connection can then be selected from the subproject connections       
and its connection profile is plotted.

</br>

---
The code below creates a form to input the FieldTwin API URL, the project id and subproject id.   
The demo requires an API access token to be entered.

In [122]:
#@title Create User Inputs
import ipywidgets as widgets
from IPython.display import display
import requests
import json

# Create the user interface

# API URL input
apiUrl = 'https://backend.qa.fieldtwin.com/API/v1.9/'
apiUrlInput = widgets.Text(
  description = 'API:',
  layout = widgets.Layout(width='25rem'),
  value = apiUrl,
  placeholder = apiUrl,
)

# API Token input
tokenInput = widgets.Password(
  description = 'Token',
  value = '',
  placeholder = 'Enter API Token',
)

# Project Id input
projectId = '-M-HHqMifhz6qskW2goc'
projectIdInput = widgets.Text(
  value = projectId,
  placeholder = projectId,
  description='Project:',
)

# Subproject Id input
subprojectId = '-M-HI0DPuadaK9GLCWzR'
subprojectIdInput = widgets.Text(
  description = 'SubProject:',
  value = subprojectId,
  placeholder = subprojectId,
)

# Load subproject button
subprojectButton = widgets.Button(
  description = 'Load SubProject', 
  button_style='info'
)

# create connection dropdown menu
connectionSelect = widgets.Dropdown()

# create the button to plot the connection profile
connectionButton = widgets.Button(
  description = 'Display Profile', 
  button_style='info'
)

# Display the input form
print('Enter the FieldTwin project settings\n')
display(
  apiUrlInput, 
  tokenInput, 
  projectIdInput, 
  subprojectIdInput, 
)

Enter the FieldTwin project settings



Text(value='https://backend.qa.fieldtwin.com/API/v1.9/', description='API:', layout=Layout(width='25rem'), pla…

Password(description='Token', placeholder='Enter API Token')

Text(value='-M-HHqMifhz6qskW2goc', description='Project:', placeholder='-M-HHqMifhz6qskW2goc')

Text(value='-M-HI0DPuadaK9GLCWzR', description='SubProject:', placeholder='-M-HI0DPuadaK9GLCWzR')

The code below sends an API GET request to load the subproject data.

In [123]:
#@title Load the subproject

# The FieldTwin subproject data
subproject = {}
connections = []
connectionList = []

# configure the subproject button action
display(subprojectButton)
@subprojectButton.on_click
def onSubprojectClick(b):
  global subproject 
  global connections
  global connectionList

  # set up the API request
  apiUrl = apiUrlInput.value
  apiToken = tokenInput.value
  projectId = projectIdInput.value
  subprojectId = subprojectIdInput.value

  if apiToken:
    print('\nLoading ...\n')

    # retrieve the subproject data using the FieldTwin API
    response = requests.get(
      f'{apiUrl}{projectId}/subProject/{subprojectId}',
      headers={
        'token': apiToken,
        'sample-every': '1',
        'simplify': 'true',
      }
    )
    subproject = response.json()

    print(f"Loaded subproject: {subproject['name']}")

    # get list of connections from the loaded subproject data
    if 'connections' in subproject:
      connections = subproject['connections']

      # extract a list of connections
      connectionList = []
      for connectionId in connections:
        connection = connections[connectionId]
        item = {
          'name': connection['params']['label'], 
          'id': connectionId
        }
        connectionList.append(item)

        # populate the connection selection menu
        connectionSelect.options = [item['name'] for item in connectionList]

  else:
    print('API access token must be entered')


Button(button_style='info', description='Load SubProject', style=ButtonStyle())

The code below displays the connection selection menu and provides a button.   
to generate the connection profile plot.

In [124]:
#@title Plot connection profile
import math
import matplotlib.pyplot as plotter
from matplotlib.pyplot import figure

# display the connection selection UI
print('Select connection')
display(connectionSelect, connectionButton)

@connectionButton.on_click
def onConnectionClick(b):
  global connections
  global connectionList

  if len(connectionList) > 0:
    # obtain the id of the selected connection
    item = connectionList[connectionSelect.index]
    connectionId = item['id']

    # get the selected connection data
    connectionData = connections[connectionId]

    # obtain the connection coordinate points
    points = []
    points.append(connectionData['fromCoordinate'])

    if 'sampled' in connectionData:
      # use the sampled connection data if it is available
      points = connectionData['sampled']
    elif 'intermediaryPoints' in connectionData:
      # otherwise use the intermediary points
      points = connectionData['intermediaryPoints']
      points.append(connectionData['toCoordinate'])

    # print(points)

    # calculate the distance vs depth profile for the connection
    profileDistance = []
    profileDepth = []
    distance = 0
    x0 = points[0]['x']
    y0 = points[0]['y']
    for point in points:
      x1 = point['x']
      y1 = point['y']
      z = point['z']
      # accumulate the along connection distance
      distance += math.hypot((x1-x0), (y1-y0))
      x0 = x1
      y0 = y1
      profileDistance.append(distance)
      profileDepth.append(z)

    # Plot the profile
    print()
    unit = subproject['coordinateUnits']
    plotter.plot(profileDistance, profileDepth, c = '#0080C0')
    plotter.title(f"{item['name']} Connection Profile")
    plotter.xlabel(f'Distance along connection ({unit})')
    plotter.ylabel(f'Depth ({unit})')
    plotter.figure(figsize=(16,4))

Select connection


Dropdown(options=(), value=None)

Button(button_style='info', description='Display Profile', style=ButtonStyle())